# **Modeling the Impact of Wheather on Water Consumption in Barcelona** 

## Data Preparation and Integration - Iteration 1

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [ ]:
data_path = '../data'

consum = pd.read_parquet(os.path.join(data_path, 'consumption/consum.parquet'))

### **Consumption Data**

In [3]:
display(consum.head())
display(consum.tail())

,POLIZA_SUMINISTRO,FECHA,CONSUMO_REAL,SECCIO_CENSAL,US_AIGUA_GEST,NUM_MUN_SGAB,NUM_DTE_MUNI,NUM_COMPLET,DATA_INST_COMP,MARCA_COMP,CODI_MODEL,DIAM_COMP
0,VECWAVDUULZDSBOP,2021-01-01,1758,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0
1,VECWAVDUULZDSBOP,2021-01-02,1854,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0
2,VECWAVDUULZDSBOP,2021-01-03,1885,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0
3,VECWAVDUULZDSBOP,2021-01-04,5676,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0
4,VECWAVDUULZDSBOP,2021-01-05,4456,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0


,POLIZA_SUMINISTRO,FECHA,CONSUMO_REAL,SECCIO_CENSAL,US_AIGUA_GEST,NUM_MUN_SGAB,NUM_DTE_MUNI,NUM_COMPLET,DATA_INST_COMP,MARCA_COMP,CODI_MODEL,DIAM_COMP
17112704,SZPMTJ7SXUODUNPY,2024-12-27,369,NaN,None,NaN,NaN,None,None,None,NaN,NaN
17112705,SZPMTJ7SXUODUNPY,2024-12-28,187,NaN,None,NaN,NaN,None,None,None,NaN,NaN
17112706,SZPMTJ7SXUODUNPY,2024-12-29,258,NaN,None,NaN,NaN,None,None,None,NaN,NaN
17112707,SZPMTJ7SXUODUNPY,2024-12-30,180,NaN,None,NaN,NaN,None,None,None,NaN,NaN
17112708,SZPMTJ7SXUODUNPY,2024-12-31,269,NaN,None,NaN,NaN,None,None,None,NaN,NaN


In [4]:
consum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17112709 entries, 0 to 17112708
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   POLIZA_SUMINISTRO  object 
 1   FECHA              object 
 2   CONSUMO_REAL       int64  
 3   SECCIO_CENSAL      float64
 4   US_AIGUA_GEST      object 
 5   NUM_MUN_SGAB       float64
 6   NUM_DTE_MUNI       float64
 7   NUM_COMPLET        object 
 8   DATA_INST_COMP     object 
 9   MARCA_COMP         object 
 10  CODI_MODEL         float64
 11  DIAM_COMP          float64
dtypes: float64(5), int64(1), object(6)
memory usage: 1.5+ GB


In [5]:
print(consum.nunique())
print("")
print(consum.isnull().sum())

POLIZA_SUMINISTRO    11797
FECHA                 1458
CONSUMO_REAL         29879
SECCIO_CENSAL          448
US_AIGUA_GEST            3
NUM_MUN_SGAB             4
NUM_DTE_MUNI             8
NUM_COMPLET           3999
DATA_INST_COMP         453
MARCA_COMP               4
CODI_MODEL              11
DIAM_COMP                3
dtype: int64

POLIZA_SUMINISTRO           0
FECHA                       0
CONSUMO_REAL                0
SECCIO_CENSAL        11312709
US_AIGUA_GEST        11312709
NUM_MUN_SGAB         11312709
NUM_DTE_MUNI         11312709
NUM_COMPLET          11312709
DATA_INST_COMP       11312709
MARCA_COMP           11312709
CODI_MODEL           11312709
DIAM_COMP            11312709
dtype: int64


In [6]:
consum['FECHA'] = pd.to_datetime(consum['FECHA'], errors='coerce')
consum = consum[consum['CONSUMO_REAL'] > 0]

drop_cols = ['US_AIGUA_GEST','NUM_MUN_SGAB','NUM_DTE_MUNI','NUM_COMPLET',
             'DATA_INST_COMP','MARCA_COMP','CODI_MODEL','DIAM_COMP']
consum = consum.drop(columns=[c for c in drop_cols if c in consum.columns])
 
display(consum.head())


,POLIZA_SUMINISTRO,FECHA,CONSUMO_REAL,SECCIO_CENSAL
0,VECWAVDUULZDSBOP,2021-01-01,1758,801903025.0
1,VECWAVDUULZDSBOP,2021-01-02,1854,801903025.0
2,VECWAVDUULZDSBOP,2021-01-03,1885,801903025.0
3,VECWAVDUULZDSBOP,2021-01-04,5676,801903025.0
4,VECWAVDUULZDSBOP,2021-01-05,4456,801903025.0


In [7]:
consum = (
    consum.groupby('FECHA')['CONSUMO_REAL']
    .sum()
    .reset_index()
    .rename(columns={'CONSUMO_REAL': 'CONSUM_DIARI'})
)

display(consum.head())

,FECHA,CONSUM_DIARI
0,2021-01-01,2882779
1,2021-01-02,3123617
2,2021-01-03,3179900
3,2021-01-04,3384061
4,2021-01-05,3390447


### **Weather Data**

In [8]:
weather_21 = pd.read_csv(os.path.join(data_path, 'weather/weather_2021_clean.csv'))
display(weather_21.head())

,DATA_LECTURA,CODI_ESTACIO,WindDir_Mean_10m,WindDir_Max_10m,Humidity_Mean,Humidity_Min,Humidity_Max,Pressure_Mean,Pressure_Min,Precipitation,Pressure_Max,Solar_Radiation_24h,Temp_Mean,Temp_Min,Temp_Max,WindSpeed_Mean_10m,WindSpeed_Max_10m
0,2021-01-01,D5,337.0,337.0,77.0,64.0,88.0,956.3,954.9,1.2,957.8,1.9,5.3,3.1,6.5,5.3,13.6
1,2021-01-01,X2,NaN,NaN,66.0,56.0,80.0,NaN,NaN,NaN,NaN,NaN,8.4,6.3,9.9,NaN,NaN
2,2021-01-01,X4,332.0,306.0,61.0,53.0,66.0,1001.0,999.6,0.1,1002.5,1.5,8.9,7.0,10.3,1.9,11.5
3,2021-01-01,X8,326.0,322.0,66.0,56.0,78.0,995.4,994.1,0.0,996.9,1.6,8.0,5.7,9.7,2.7,10.2
4,2021-01-02,D5,335.0,326.0,71.0,54.0,84.0,956.8,955.1,0.0,959.5,7.2,3.2,0.2,7.2,7.3,14.4


In [9]:
# Concatenate weather data 
weather_22 = pd.read_csv(os.path.join(data_path, 'weather/weather_2022_clean.csv'))
weather_23 = pd.read_csv(os.path.join(data_path, 'weather/weather_2023_clean.csv'))
weather_24 = pd.read_csv(os.path.join(data_path, 'weather/weather_2024_clean.csv'))

weather = pd.concat([weather_21, weather_22, weather_23, weather_24], ignore_index=True)
display(weather.head())
display(weather.tail())

,DATA_LECTURA,CODI_ESTACIO,WindDir_Mean_10m,WindDir_Max_10m,Humidity_Mean,Humidity_Min,Humidity_Max,Pressure_Mean,Pressure_Min,Precipitation,Pressure_Max,Solar_Radiation_24h,Temp_Mean,Temp_Min,Temp_Max,WindSpeed_Mean_10m,WindSpeed_Max_10m
0,2021-01-01,D5,337.0,337.0,77.0,64.0,88.0,956.3,954.9,1.2,957.8,1.9,5.3,3.1,6.5,5.3,13.6
1,2021-01-01,X2,NaN,NaN,66.0,56.0,80.0,NaN,NaN,NaN,NaN,NaN,8.4,6.3,9.9,NaN,NaN
2,2021-01-01,X4,332.0,306.0,61.0,53.0,66.0,1001.0,999.6,0.1,1002.5,1.5,8.9,7.0,10.3,1.9,11.5
3,2021-01-01,X8,326.0,322.0,66.0,56.0,78.0,995.4,994.1,0.0,996.9,1.6,8.0,5.7,9.7,2.7,10.2
4,2021-01-02,D5,335.0,326.0,71.0,54.0,84.0,956.8,955.1,0.0,959.5,7.2,3.2,0.2,7.2,7.3,14.4


,DATA_LECTURA,CODI_ESTACIO,WindDir_Mean_10m,WindDir_Max_10m,Humidity_Mean,Humidity_Min,Humidity_Max,Pressure_Mean,Pressure_Min,Precipitation,Pressure_Max,Solar_Radiation_24h,Temp_Mean,Temp_Min,Temp_Max,WindSpeed_Mean_10m,WindSpeed_Max_10m
5747,2024-12-30,X4,180.0,194.0,64.0,44.0,77.0,1025.7,1024.8,0.0,1026.9,9.0,10.7,6.7,15.1,0.6,3.7
5748,2024-12-30,X8,288.0,189.0,71.0,37.0,94.0,1019.9,1019.0,0.0,1021.0,8.4,9.0,4.8,15.4,0.6,4.0
5749,2024-12-31,D5,137.0,47.0,86.0,65.0,97.0,981.8,980.8,0.0,982.9,6.8,8.6,7.4,12.2,1.9,5.0
5750,2024-12-31,X4,53.0,5.0,72.0,53.0,83.0,1027.2,1026.3,0.0,1028.4,7.0,10.9,7.0,14.4,0.8,3.8
5751,2024-12-31,X8,348.0,83.0,87.0,52.0,100.0,1021.4,1020.5,0.1,1022.6,6.7,9.2,4.4,14.4,0.9,3.3


In [10]:
weather['DATA_LECTURA'] = pd.to_datetime(weather['DATA_LECTURA'], errors='coerce')

weather = (
    weather
    .groupby('DATA_LECTURA')
    .mean(numeric_only=True)
    .reset_index()
)

weather = weather.rename(columns={'DATA_LECTURA': 'FECHA'})
weather = weather.round(2)

display(weather.head())

,FECHA,WindDir_Mean_10m,WindDir_Max_10m,Humidity_Mean,Humidity_Min,Humidity_Max,Pressure_Mean,Pressure_Min,Precipitation,Pressure_Max,Solar_Radiation_24h,Temp_Mean,Temp_Min,Temp_Max,WindSpeed_Mean_10m,WindSpeed_Max_10m
0,2021-01-01,331.67,321.67,67.50,57.25,78.00,984.23,982.87,0.43,985.73,1.67,7.65,5.52,9.10,3.30,11.77
1,2021-01-02,324.33,316.00,61.25,46.75,72.75,985.00,983.40,0.00,987.57,7.00,5.90,2.85,9.40,4.10,11.13
2,2021-01-03,302.67,316.67,49.25,33.25,69.25,988.83,987.20,0.00,990.27,8.57,5.32,2.58,9.80,3.30,11.27
3,2021-01-04,309.33,301.00,59.25,36.75,75.25,988.17,985.90,0.00,989.67,8.90,5.60,2.28,10.30,2.47,9.90
4,2021-01-05,299.33,322.00,65.00,46.75,78.50,989.37,987.67,0.00,991.10,7.10,4.95,2.08,9.35,2.53,7.83


### **Integration**

In [11]:
df = pd.merge(
    consum,
    weather,
    on='FECHA',
    how='inner'
)

df.head()

,FECHA,CONSUM_DIARI,WindDir_Mean_10m,WindDir_Max_10m,Humidity_Mean,Humidity_Min,Humidity_Max,Pressure_Mean,Pressure_Min,Precipitation,Pressure_Max,Solar_Radiation_24h,Temp_Mean,Temp_Min,Temp_Max,WindSpeed_Mean_10m,WindSpeed_Max_10m
0,2021-01-01,2882779,331.67,321.67,67.50,57.25,78.00,984.23,982.87,0.43,985.73,1.67,7.65,5.52,9.10,3.30,11.77
1,2021-01-02,3123617,324.33,316.00,61.25,46.75,72.75,985.00,983.40,0.00,987.57,7.00,5.90,2.85,9.40,4.10,11.13
2,2021-01-03,3179900,302.67,316.67,49.25,33.25,69.25,988.83,987.20,0.00,990.27,8.57,5.32,2.58,9.80,3.30,11.27
3,2021-01-04,3384061,309.33,301.00,59.25,36.75,75.25,988.17,985.90,0.00,989.67,8.90,5.60,2.28,10.30,2.47,9.90
4,2021-01-05,3390447,299.33,322.00,65.00,46.75,78.50,989.37,987.67,0.00,991.10,7.10,4.95,2.08,9.35,2.53,7.83


In [12]:
df.to_csv(os.path.join(data_path, 'consumption_weather.csv'), index=False)
print("Integrated dataset ready for modeling.")

Integrated dataset ready for modeling.
